In [1]:
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt

# mask初级使用

In [2]:
# 读取图像并创建对应掩膜
img = cv2.imread("./dataset/background.jpg")

## method 1

In [3]:
mask = np.zeros(img.shape, dtype=np.uint8)

In [4]:
print(img.shape)

(690, 690, 3)


原始图像的尺寸为（690,690），指定矩形ROI

In [5]:
mask[300:690, : , : ] = 255 #白色部分是想要保留的部分

In [6]:
mask_img = cv2.bitwise_and(img, mask)

In [7]:
cv2.imshow("raw image", img)
cv2.imshow("mask", mask)
cv2.imshow("masked image", mask_img)

cv2.waitKey(0)
cv2.destroyAllWindows()

## method 2
若图像较大，使用method 2比method 1运算速度更快

In [8]:
h, w, c = img.shape # 获取高度,宽度,通道数
mask_2 = np.zeros((h, w), dtype=np.uint8)
mask_2[300:690, : ] = 200 # 将ROI设置为不为0

In [9]:
mask_img_2 = cv2.bitwise_and(img, img, mask=mask_2) # 自身与运算得到原图像

In [10]:
cv2.imshow("masked image", mask_img_2)

cv2.waitKey(0)
cv2.destroyAllWindows()

# 识别不规则区域


演示案例：加载logo图片，去除logo图片中的背景，并用logo覆盖目标图像img的相应位置。



<div style="display: flex; align-items: center; justify-content: space-around;">
    <img src="dataset/logo.png" alt="logo" style="max-width: 49%;">
    <img src="dataset/background.jpg" alt="img" style="max-width: 49%;">
</div>

In [85]:
# 加载logo图片
raw_logo = cv2.imread('dataset/logo.png')
assert raw_logo is not None, "file could not be read, check with os.path.exists()"

In [92]:
# 读取添加logo的背景图
img = cv2.imread("./dataset/background.jpg")
assert img is not None, "file could not be read, check with os.path.exists()"

In [93]:
img.shape

(690, 690, 3)

In [94]:
raw_logo.shape

(2048, 2048, 3)

In [95]:
# 缩小图像尺寸
scale = 1/4
logo = cv2.resize(raw_logo, dsize=None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)

In [96]:
logo.shape

(512, 512, 3)

In [97]:
# 将logo放置在背景图的左上角
rows,cols,channels = logo.shape
roi = img[0:rows, 0:cols]

In [98]:
cv2.imshow('roi',roi)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Step1：转换为二值图

1. 将输入图像转换为灰度图 `cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)`
2. 通过`cv2.threshold`将感兴趣的部分设置为白色（像素值255）前景，不感兴趣的部分统一设置为黑色（像素值0）背景

In [99]:
# 创建logo的掩膜
logo_gray = cv2.cvtColor(logo,cv2.COLOR_BGR2GRAY)

### cv2.threshold

The function is typically used to get a bi-level (binary) image out of a grayscale image or for removing a noise, that is, filtering out pixels with too small or too large values. There are several types of thresholding supported by the function. They are determined by type parameter.
- maxval: maximum value to use with the `THRESH_BINARY` and `THRESH_BINARY_INV` thresholding types.
- retval: 实际使用的阈值
- dst: 处理后的图像

In [100]:
cv2.imshow('logo_gray',logo_gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

注意到在创建掩膜时，白色区域是我们感兴趣的区域（ROI）。即，mask_inv是logo对应的掩膜。在下一章节我们考虑获取更加紧凑的表示，即去除周围的黑色边框。

In [111]:
# 获取二值灰度图像
ret, mask = cv2.threshold(logo_gray, 230, 255, cv2.THRESH_BINARY) #将原logo中的背景色设置为白色(255)，logo部分设置为黑色
ret, mask_inv = cv2.threshold(logo_gray, 230, 255, cv2.THRESH_BINARY_INV) #将原logo中的背景色设置为黑色(0)，logo部分设置为白色
# mask_inv = cv2.bitwise_not(mask)

In [102]:
mask_inv.shape

(512, 512)

In [103]:
cv2.namedWindow('mask', cv2.WINDOW_NORMAL) 
cv2.resizeWindow('mask', width=600, height=600)
cv2.imshow('mask',mask_inv)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [105]:
# 保留roi中与logo图像背景重叠的部分
img_bg = cv2.bitwise_and(roi,roi,mask = mask)

# 提取logo图像中的logo部分
logo_fg = cv2.bitwise_and(logo,logo,mask = mask_inv)

# 将logo放置在roi中
dst = cv2.add(img_bg,logo_fg)
img[0:rows, 0:cols ] = dst

In [106]:
cv2.imshow('image_logo',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Step2: 获取前景的边缘

1. 使用`cv2.findNonZero(binary_image)`,`cv2.findContours(binary_image)`方法帮助定位前景的位置
2. 使用`cv2.boundingRect()`找到前景区域的最小外接矩形

In [120]:
# 读取添加logo的背景图
img = cv2.imread("./dataset/background.jpg")
assert img is not None, "file could not be read, check with os.path.exists()"

上一章节添加logo时，由于logo图片的背景占比过大，原本想要添加到左上角的logo实际添加到了img图像的中部。因此需要对相应的掩膜mask_inv进行裁切，去除周围的黑色边框。

In [114]:
cv2.namedWindow('mask', cv2.WINDOW_NORMAL) 
cv2.resizeWindow('mask', width=600, height=600)
cv2.imshow('mask',mask_inv)
cv2.waitKey(0)
cv2.destroyAllWindows()

## cv2.findNonZero()

In [116]:
# 查找非零元素的位置
non_zero_points = cv2.findNonZero(mask_inv)
print(non_zero_points.shape)

# 将坐标数组重新塑形为(N, 2),方便输出
non_zero_points = non_zero_points.reshape((-1, 2))

# 输出结果
# for point in non_zero_points:
#     x, y = point.ravel()
#     print(f'Point found at ({x}, {y})')

(28880, 1, 2)


## cv2.boundingRect()

In [117]:
x, y, w, h = cv2.boundingRect(cv2.findNonZero(mask_inv))
cropped_mask_inv = mask_inv[y:y+h, x:x+w]

In [118]:
cv2.imshow('cropped_mask',cropped_mask_inv)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [119]:
cropped_mask = cv2.bitwise_not(cropped_mask_inv)

In [122]:
# 将logo放置在背景图的左上角
rows,cols = cropped_mask.shape
roi = img[0:rows, 0:cols]

In [123]:
# 保留roi中与logo图像背景重叠的部分
img_bg = cv2.bitwise_and(roi,roi,mask = cropped_mask)

# 提取logo图像中的logo部分
cropped_logo = logo[y:y+h, x:x+w]
logo_fg = cv2.bitwise_and(cropped_logo,cropped_logo,mask = cropped_mask_inv)

# 将logo放置在roi中
dst = cv2.add(img_bg,logo_fg)
img[0:rows, 0:cols ] = dst

In [124]:
cv2.imshow('image_logo',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 项目：识别漫画的画框

## cv2.findContours()

`image, contours, hierarchy = cv2.findContours(image, mode, method)`

用于在二值图像中查找物体的轮廓信息。
- 二值图像通常经过预处理：对原始图像进行灰度化、滤波、阈值处理等等
- 原理：遍历二值图像的每个像素，识别出相邻的、具有相同像素值（通常为白色或黑色）的像素集合，从而确定物体的轮廓。
- mode参数
    - cv2.RETR_EXTERNAL     表示只检测外轮廓
    - cv2.RETR_LIST         检测的轮廓不建立等级关系
    - cv2.RETR_CCOMP        建立两个等级的轮廓，上面的一层为外边界，里面的一层为内孔的边界信息。如果内孔内还有一个连通物体，这个物体的边界也在顶层。
    - cv2.RETR_TREE         建立一个等级树结构的轮廓。
- method参数
    - cv2.CHAIN_APPROX_NONE   存储所有的轮廓点，相邻的两个点的像素位置差不超过1，即max（abs（x1-x2），abs（y2-y1））==1
    - cv2.CHAIN_APPROX_SIMPLE 压缩水平方向，垂直方向，对角线方向的元素，只保留该方向的终点坐标，例如一个矩形轮廓只需4个点来保存轮廓信息
    - cv2.CHAIN_APPROX_TC89_L1，CV_CHAIN_APPROX_TC89_KCOS 使用teh-Chinl chain近似算法
- 返回值：
    - image：只包含轮廓点的原始尺寸图像，通常不关注
    - contours：Python列表，每个元素是一个轮廓坐标点集
    - hierarchy：一个包含轮廓层次信息的数组，表示轮廓之间的嵌套关系
- 嵌套关系：每个轮廓contours[i]对应4个说明轮廓关系的元素[Next, Previous, First_Child, Parent]
    - Next：后一个轮廓的索引编号。
    - Previous：前一个轮廓的索引编号。
    - First_Child：第 1 个子轮廓的索引编号。
    - Parent：父轮廓的索引编号。
    - 对应关系为空时，取值-1

In [2]:
# 读取图像并转换为灰度图像
image = cv2.imread('./dataset/001-22.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
img_h,img_w = gray.shape

In [3]:
# 缩小图像尺寸
# scale = 1/4
# gray_rs = cv2.resize(gray, dsize=None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
# h, w = gray_rs.shape

In [4]:
scale = 0.4
cv2.namedWindow('image', cv2.WINDOW_NORMAL) 
cv2.resizeWindow('image', width=int(img_w*scale), height=int(img_h*scale))
cv2.imshow('image',gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
# 应用阈值处理得到二值图像
# 检测对象为白色，背景为黑色
_, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)

In [6]:
# 存储图像
# cv2.imwrite('./output/ocr_image.jpg', binary)

In [7]:
# morphological tranformation
# kernel = np.ones([3,3], dtype=np.int8)
# binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

In [8]:
scale = 0.4
cv2.namedWindow('image', cv2.WINDOW_NORMAL) 
cv2.resizeWindow('image', width=int(img_w*scale), height=int(img_h*scale))
cv2.imshow('image',binary)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
# 查找所有轮廓，并建立等级关系
contours, hierarchy = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# 绘制轮廓
image_contours = cv2.drawContours(image.copy(), contours, -1, (0, 255, 0), 2)

In [10]:
# 显示结果
cv2.namedWindow('Contours', cv2.WINDOW_NORMAL) 
cv2.resizeWindow('Contours', width=int(img_w*scale), height=int(img_h*scale))
cv2.imshow('Contours', image_contours)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
# 获取轮廓的个数
print(len(contours))

5870


In [12]:
# 获取轮廓层次关系

hierarchy = hierarchy.reshape(-1,4)
print(hierarchy.shape)

(5870, 4)


In [13]:
# 查找最外层轮廓的子轮廓
parent_contour_index = 0
 
# 找到父轮廓并计算面积
parent_contour = contours[parent_contour_index]
 
# 遍历所有轮廓，找到子轮廓
child_contours = []
for i, contour in enumerate(contours):
    # 检查轮廓的父轮廓是否是我们要找的那个
    if hierarchy[i][3] == parent_contour_index:
        x, y, w, h = cv2.boundingRect(contour)
        child_contours.append(contour)
 
# 绘制子轮廓
contour_img = image.copy()
for contour in child_contours:
    cv2.drawContours(contour_img, [contour], -1, (0, 255, 0), 2)

In [14]:
print(len(child_contours))

99


In [15]:
# 显示图像
cv2.namedWindow('Contours', cv2.WINDOW_NORMAL) 
cv2.resizeWindow('Contours', width=int(img_w*scale), height=int(img_h*scale))
cv2.imshow('Contours', contour_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 筛选panels

### 长宽占比

In [16]:
rects = [] # 存储轮廓最小边框坐标
ratio = 1/10 # 定义panels长宽占比
for contour in child_contours:
    x, y, w, h = cv2.boundingRect(contour)
    # 判断是否为panel
    if (w < img_w*ratio) or (h < img_h*ratio):
        pass
    else:
        rect = np.array([x, y, w, h])
        rects.append(rect)

In [17]:
panel_img = image.copy()
for i in range(len(rects)):
    x, y, w, h = rects[i]
    # 在图像上绘制矩形
    cv2.rectangle(panel_img, (x, y), (x + w, y + h), (0,0,255), 2)

In [18]:
print(len(rects))

5


In [19]:
# 显示图像
cv2.namedWindow('Contours', cv2.WINDOW_NORMAL) 
cv2.resizeWindow('Contours', width=int(img_w*scale), height=int(img_h*scale))
cv2.imshow('Contours', panel_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [20]:
test_img = image.copy()
for contour in child_contours:
    # 计算轮廓的近似值，epsilon值越大，近似度越高，越接近真实轮廓
    epsilon = 10**(-4) * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)

    # 绘制轮廓
    cv2.drawContours(test_img, [approx], 0, (0, 255, 0), 3)

In [21]:
# 显示图像
cv2.namedWindow('Contours', cv2.WINDOW_NORMAL) 
cv2.resizeWindow('Contours', width=int(img_w*scale), height=int(img_h*scale))
cv2.imshow('Contours', test_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 计算面积占比

In [22]:
total_area = cv2.contourArea(parent_contour)
threshold_area_ratio = 1/100

In [23]:
contours = []
rects = []
for contour in child_contours:
    contour_area = cv2.contourArea(contour)
    area_ratio = contour_area / total_area
    if area_ratio > threshold_area_ratio:
        contours.append(contour)
        rect = cv2.boundingRect(contour)
        rects.append(rect)

In [24]:
print(len(contours))

5


In [25]:
panel_img = image.copy()
for i in range(len(rects)):
    x, y, w, h = rects[i]
    # 在图像上绘制矩形
    cv2.rectangle(panel_img, (x, y), (x + w, y + h), (0,0,255), 2)

In [26]:
# 显示图像
cv2.namedWindow('Contours', cv2.WINDOW_NORMAL) 
cv2.resizeWindow('Contours', width=int(img_w*scale), height=int(img_h*scale))
cv2.imshow('Contours', panel_img)
cv2.waitKey(0)
cv2.destroyAllWindows()